In [1]:
import netCDF4 as nc
import scipy.io as sio
import os,sys,fnmatch,time
from scipy import interpolate
import numpy as np
import numpy.ma as ma
import math
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm
import cmocean as cmo

from salishsea_tools.nc_tools import scDataset
from salishsea_tools import geo_tools

import sys
sys.path.append('/ocean/imachuca/Canyons/mackenzie_canyon/tools/')
import functions_plot

sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation/')
import general_functions

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

(cmapmc, norm) = functions_plot.Colormap()
from matplotlib.colors import ListedColormap
cmapmc_r = ListedColormap(cmapmc.colors[::-1])
from matplotlib.colors import LinearSegmentedColormap
colour_list = ["#c8274c","#f25546","#F06543","#e96e33",
               "#f0b038","#FFE74C","#69b944","#72b286",
               "#69b0bc","#619ee4","#4b5bbb"][::-1]
cmapIAM = LinearSegmentedColormap.from_list('mycmap', colour_list, N=500, gamma=1)

## Functions for observations

In [27]:
data = sio.loadmat('./MackenzieTransects.mat')

struct_north = data['stn']
struct_south = data['sts']

dtype_north = struct_north.dtype
dtype_south = struct_south.dtype

data_north = {n: struct_north[n][0, 0] for n in dtype_north.names}
data_south = {n: struct_south[n][0, 0] for n in dtype_south.names}

obs_lat_s = data_south['lat_adcp'][0,:]
obs_lon_s = data_south['lon_adcp'][0,:]
obs_dist_s = data_south['dist_adcp'][0,:]
obs_z_s = data_south['z_adcp'][:, 0]
obs_u_s = data_south['u'][:]
obs_v_s = data_south['v'][:]


obs_lat_n = data_north['lat_adcp'][0,:]
obs_lon_n = data_north['lon_adcp'][0,:]
obs_dist_n = data_north['dist_adcp'][0,:]
obs_z_n = data_north['z_adcp'][:, 0]
obs_u_n = data_north['u'][:]
obs_v_n = data_north['v'][:]

In [37]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

def get_obs_mean(vel_s, vel_n, z, dep1, dep2):
    
    ind1 = find_nearest(z, dep1)
    ind2 = find_nearest(z, dep2)
    
    vel_mean_s = np.mean(vel_s[ind1:ind2, :], axis=0)

    vel_mean_n = np.mean(vel_n[ind1:ind2, :], axis=0)
    
    return vel_mean_s, vel_mean_n, z[ind1], z[ind2]

## Functions for model results

In [49]:
def get_deps(dirname, velocity):
    fname = "1_MCKNZ_1h_20170101_201701*"
    
    if velocity == 'U':
        name_grid = 'grid_U'
        name_depth = 'depthu'
            
    elif velocity == 'V':
        name_grid = 'grid_V'
        name_depth = 'depthv'
        
    files = general_functions.get_files(dirname, fname, name_grid)
    with scDataset(files) as ds:
        depth = ds.variables[name_depth][:]
        
    return depth

In [58]:
def get_vars_U(dirname, dep1, dep2, time_start, time_end, velocity):
    fname = "1_MCKNZ_1h_20170101_201701*"
    y,x = slice(1,-1,None), slice(1,-1,None)
    
    # --------------------------------------------------------
    
    depth = get_deps(dirname, velocity)
    dep_start = find_nearest(depth, dep1)
    dep_end = (find_nearest(depth, dep2)) + 1
    
    # --------------------------------------------------------
    
    if velocity == 'U':
        name_grid = 'grid_U'
        name_depth = 'depthu'
        name_vel = 'vozocrtx'
        name_mask = 'umask'
        
    elif velocity == 'V':
        name_grid = 'grid_V'
        name_depth = 'depthv'
        name_vel = 'vomecrty'
        name_mask = 'vmask'
        
    # --------------------------------------------------------
        
    files = general_functions.get_files(dirname, fname, name_grid)    
    with scDataset(files) as ds:
        depth = ds.variables[name_depth][dep_start:dep_end]
        vel0 = ds.variables[name_vel][time_start:time_end, dep_start:dep_end, y, x]
        nav_lat = ds.variables['nav_lat'][y, x]
        nav_lon = ds.variables['nav_lon'][y, x]
            
    with nc.Dataset(os.path.join(dirname, '1_mesh_mask.nc'), 'r') as dsM:
        mask0 = dsM.variables[name_mask][0, dep_start:dep_end, y, x]
        
    # --------------------------------------------------------
        
    mask = np.tile(mask0, (len(vel0), 1, 1, 1))
    vel = np.ma.array(vel0, mask=1 - mask)
    
    return vel, depth, nav_lat, nav_lon

In [51]:
def get_vars_mean(dirname, dep_start, dep_end, time_start, time_end):
    vozocrtx, vomecrty, depthu, depthv, nav_lat, nav_lon = get_vars(dirname, dep_start, dep_end, time_start, time_end)
    vozocrtx_mean = np.mean(np.mean(vozocrtx, axis=0), axis=0)
    vomecrty_mean = np.mean(np.mean(vomecrty, axis=0), axis=0)
    return vozocrtx_mean, vomecrty_mean, depthu, depthv, nav_lat, nav_lon

## Test

In [54]:
dep1, dep2 = 21, 39

time_start = 10
time_end = 12

In [53]:
vel_mean_s, vel_mean_n, obs_z_start, obs_z_end = get_obs_mean(obs_u_s, obs_u_n, obs_z_s, dep1, dep2)

obs_z_start, obs_z_end

(20.651493617210576, 39.445346032928747)

In [59]:
dirname = '/ocean/imachuca/Canyons/results_mackenzie/initial_collection/real_base/'

vel, depth, nav_lat, nav_lon = get_vars_U(dirname, dep1, dep2, time_start, time_end, 'U')

## Figures